### Imports

In [3]:
import pandas as pd
import os
import langdetect
from functools import lru_cache
from deep_translator import GoogleTranslator
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 42

In [4]:
import sys
sys.path.append(os.path.abspath('../../src'))

from helper_functions.path_resolver import DynamicPathResolver

### Paths

In [5]:
dpr = DynamicPathResolver(marker="README.md")

data_mail_dir = dpr.path.data.raw.data_mail.sets._path

train_paths_curated = [
    dpr.path.data.raw.data_mail.curated.CEAS_08_csv,
    dpr.path.data.raw.data_mail.curated.TREC_07_csv
]

test_paths_curated = [
    dpr.path.data.raw.data_mail.curated.Nazario_5_csv,
    dpr.path.data.raw.data_mail.curated.SpamAssasin_csv
]

paths_own = [
    dpr.path.data.raw.data_mail.own.mails_combined_csv,
   # dpr.path.data.raw.data_mail.own.mails_jannis.jannis_mail_csv
]

Project Root: c:\Users\ilian\Documents\Projects\git_projects\university\phishing_bert


### Functions

In [6]:
translator = GoogleTranslator(source="en", target="de")
tqdm.pandas()

In [7]:
def detect_language(text):
    try:
        return langdetect.detect(str(text))
    except Exception:
        return "unknown"
    

def translate_to_de(text):
    try:
        return translator.translate(text)
    except Exception:
        return text

In [8]:
def build_balanced_set(base_file, out_file, total_size, language="en"):
    per_class = total_size // 2

    df = pd.read_csv(base_file)
    df = df[df["label"].isin([0, 1]) & (df["language"] == language)]

    legit_samp = df[df["label"] == 0].sample(n=per_class, random_state=42)
    phish_samp = df[df["label"] == 1].sample(n=per_class, random_state=42)

    balanced = pd.concat([legit_samp, phish_samp], ignore_index=True)
    balanced.fillna({"subject": "", "body": ""}, inplace=True)
    balanced.to_csv(out_file, index=False)

In [9]:
def translate_entire_dataset(eng_file, out_file):
    df = pd.read_csv(eng_file)
    print(f"Translating dataset: {eng_file}")
    
    df['subject'] = [translate_to_de(text) for text in tqdm(df['subject'], desc="Translating subjects")]
    df['body'] = [translate_to_de(text) for text in tqdm(df['body'], desc="Translating bodies")]
    df['language'] = "de"
    df.to_csv(out_file, index=False)

    print(f"Translated dataset saved: {out_file}")

In [10]:
def sample_from_existing_datasets(eng_file, de_file, out_file, english_size, german_size):
    for file, desc in zip([eng_file, de_file], ["English", "German"]):
        tqdm.write(f"Processing {desc} file: {file}")
        
    df_eng = pd.read_csv(eng_file)
    df_de = pd.read_csv(de_file)

    # Split English into phishing (1) and legit (0)
    eng_phishing = df_eng[df_eng["label"] == 1]
    eng_legit = df_eng[df_eng["label"] == 0]

    # Split German into phishing (1) and legit (0)
    de_phishing = df_de[df_de["label"] == 1]
    de_legit = df_de[df_de["label"] == 0]

    # First half phishing, second half legit
    eng_phishing_sample = eng_phishing.iloc[:english_size // 2]  
    eng_legit_sample = eng_legit.iloc[:english_size // 2]

    de_phishing_sample = de_phishing.iloc[-german_size // 2:]  
    de_legit_sample = de_legit.iloc[-german_size // 2:]

    # Combine
    combined = pd.concat([eng_phishing_sample, eng_legit_sample, de_phishing_sample, de_legit_sample], ignore_index=True)
    combined.to_csv(out_file, index=False)

    print(f"Built multilingual dataset: {out_file}")

    return combined

In [11]:
def verify(df, name):
    print(f"\n{name}, Rows: {len(df)}")
    print("----------------------------------------")
    
    class_counts = df["label"].value_counts().to_dict()
    lang_counts = df["language"].value_counts().to_dict()
    grouped = df.groupby(["label", "language"]).size().to_dict()

    print(f"Class Distribution: {class_counts}")
    print(f"Language Distribution: {lang_counts}")
    print(f"Detailed (Class, Language) Distribution: {grouped}")
    print("----------------------------------------")

In [12]:
def add_lang_and_create_base(file_paths, out_file):
    dfs = []
    for path in tqdm(file_paths, desc="Preprocessing and combining files", unit="file"):
        df = pd.read_csv(path)
        df["language"] = df["subject"].progress_apply(detect_language)
        dfs.append(df)
    combined = pd.concat(dfs, ignore_index=True)
    combined.to_csv(out_file, index=False)
    print(f"Combined preprocessed file saved: {out_file}")
    return out_file

In [2]:
from sklearn.model_selection import train_test_split

def balance_and_split_dataset(base_file, train_file, test_file, test_size=0.2, random_state=42):
    df = pd.read_csv(base_file)
    df = df[df["label"].isin([0, 1])]
    
    class_counts = df["label"].value_counts()
    print("Original dataset distribution:\n", class_counts)

    min_class_size = min(class_counts)
    legit_sample = df[df["label"] == 0]  
    phish_sample = df[df["label"] == 1].sample(n=min_class_size, random_state=random_state)  

    df_balanced = pd.concat([legit_sample, phish_sample], ignore_index=True)
    print("Balanced dataset distribution:\n", df_balanced["label"].value_counts())

    train_df, test_df = train_test_split(
        df_balanced, 
        test_size=test_size, 
        stratify=df_balanced["label"], 
        random_state=random_state
    )

    train_df.to_csv(train_file, index=False)
    test_df.to_csv(test_file, index=False)

    print(f"Train set saved: {train_file}, Size: {len(train_df)}")
    print(f"Test set saved: {test_file}, Size: {len(test_df)}")

### Language adding

##### Base paths

In [ ]:
curated_train_base = os.path.join(data_mail_dir, "mails_combined.csv")
curated_test_base  = os.path.join(data_mail_dir, "curated_test_base.csv")

##### Add language col to bases

In [ ]:
add_lang_and_create_base(train_paths_curated, curated_train_base)
add_lang_and_create_base(test_paths_curated, curated_test_base)

##### Verify 

In [ ]:
df_curated_train_base = pd.read_csv(curated_train_base)
verify(df_curated_train_base, "Curated Train Base")

df_curated_test_base = pd.read_csv(curated_test_base)
verify(df_curated_test_base, "Curated Test Base")

### Create Curated Train & Test Sets

##### 1. BERT – English Only

In [ ]:
build_balanced_set(
    base_file=curated_train_base,
    out_file=os.path.join(data_mail_dir, "english_curated_train.csv"),
    total_size=20000
)

In [ ]:
build_balanced_set(
    base_file=curated_test_base,
    out_file=os.path.join(data_mail_dir, "english_curated_test.csv"),
    total_size=4000
)

##### 2. BERT – German Only

In [ ]:
translate_entire_dataset(
    eng_file=os.path.join(data_mail_dir, "english_curated_verification.csv"),
    out_file=os.path.join(data_mail_dir, "german_curated_verification.csv")
)

In [ ]:
translate_entire_dataset(
    eng_file=os.path.join(data_mail_dir, "english_curated_train.csv"),
    out_file=os.path.join(data_mail_dir, "german_curated_train.csv")
)

In [ ]:
translate_entire_dataset(
    eng_file=os.path.join(data_mail_dir, "english_curated_test.csv"),
    out_file=os.path.join(data_mail_dir, "german_curated_test.csv")
)

##### 3. BERT – Multilingual

In [ ]:
sample_from_existing_datasets(
    eng_file=os.path.join(data_mail_dir, "english_curated_verification.csv"),
    de_file=os.path.join(data_mail_dir, "german_curated_verification.csv"),
    out_file=os.path.join(data_mail_dir, "multilingual_curated_verification.csv"),
    english_size=2000,
    german_size=2000
)

In [ ]:
sample_from_existing_datasets(
    eng_file=os.path.join(data_mail_dir, "english_curated_train.csv"),
    de_file=os.path.join(data_mail_dir, "german_curated_train.csv"),
    out_file=os.path.join(data_mail_dir, "multilingual_curated_train.csv"),
    english_size=10000,
    german_size=10000
)

In [ ]:
sample_from_existing_datasets(
    eng_file=os.path.join(data_mail_dir, "english_curated_test.csv"),
    de_file=os.path.join(data_mail_dir, "german_curated_test.csv"),
    out_file=os.path.join(data_mail_dir, "multilingual_curated_test.csv"),
    english_size=2000,
    german_size=2000
)

### Sample non overlapping (Verification)

##### Sample

In [ ]:
def sample_non_overlapping(train_curated_paths, used_train_file, output_file, sample_size=4000):
    # Load and concatenate datasets
    df_list = [pd.read_csv(path) for path in train_curated_paths]
    full_data = pd.concat(df_list, ignore_index=True)
    used_data = pd.read_csv(used_train_file)

    # Drop duplicates
    full_data = full_data.drop_duplicates(subset=["subject", "body"])
    used_data = used_data.drop_duplicates(subset=["subject", "body"])

    # Detect language
    full_data["language"] = full_data["body"].apply(detect_language)

    # Keep only English samples
    full_data = full_data[full_data["language"] == "en"]

    # Merge with used data to find non-overlapping samples
    merged = full_data.merge(
        used_data,
        on=["subject", "body"],
        how="left",
        indicator=True,
        suffixes=("", "_drop")
    )

    new_data = (
        merged
        .query("_merge == 'left_only'")
        .drop(columns=["_merge"] + [col for col in merged.columns if col.endswith("_drop")])
    )

    # Sample a balanced dataset
    per_class = sample_size // 2
    legit_samples = new_data[new_data["label"] == 0].sample(n=per_class, random_state=42)
    phish_samples = new_data[new_data["label"] == 1].sample(n=per_class, random_state=42)
    balanced_sample = pd.concat([legit_samples, phish_samples], ignore_index=True)

    # Save
    balanced_sample.to_csv(output_file, index=False)

    # Plot class distribution
    class_dist = balanced_sample["label"].value_counts()
    print("\nClass Distribution:\n", class_dist)

    plt.figure(figsize=(6, 4))
    sns.barplot(x=class_dist.index, y=class_dist.values)
    plt.xlabel("Class")
    plt.ylabel("Count")
    plt.title("Class Distribution")
    plt.show()

    return balanced_sample


In [ ]:
used_train_file = os.path.join(data_mail_dir, "english_curated_train.csv")
output_file = os.path.join(data_mail_dir, "english_curated_verification.csv")

sample_non_overlapping(train_paths_curated, used_train_file, output_file)

##### Check duplicates

In [ ]:
def check_duplicates(csv_path1, csv_path2):
    df1 = pd.read_csv(csv_path1)
    df2 = pd.read_csv(csv_path2)

    combined_df = pd.concat([df1, df2])
    duplicates = combined_df.duplicated(keep=False)
    num_duplicates = duplicates.sum()

    print(f"Number of duplicate rows across both CSVs: {num_duplicates}")
    return num_duplicates

In [ ]:
csv_file1 = dpr.path.data.raw.data_mail.sets.english_curated_test_csv 
csv_file2 = dpr.path.data.raw.data_mail.sets.english_curated_verification_csv

In [ ]:
num_duplicates = check_duplicates(csv_file1, csv_file2)

### Create own

In [54]:
own_base = os.path.join(data_mail_dir, "own_base.csv")
own_train_base = os.path.join(data_mail_dir, "own_train_base.csv")
own_test_base  = os.path.join(data_mail_dir, "own_test_base.csv")

In [21]:
add_lang_and_create_base(paths_own, own_base)

Preprocessing and combining files:   0%|          | 0/1 [00:00<?, ?file/s]

  0%|          | 0/8162 [00:00<?, ?it/s]

Combined preprocessed file saved: c:\Users\ilian\Documents\Projects\git_projects\university\phishing_bert\data\raw\data_mail\sets\own_base.csv


'c:\\Users\\ilian\\Documents\\Projects\\git_projects\\university\\phishing_bert\\data\\raw\\data_mail\\sets\\own_base.csv'

In [50]:
balance_and_split_dataset(own_base, own_train_base, own_test_base)

Original dataset distribution:
 label
1    4543
0    3609
Name: count, dtype: int64
Balanced dataset distribution:
 label
0    3609
1    3609
Name: count, dtype: int64
Train set saved: c:\Users\ilian\Documents\Projects\git_projects\university\phishing_bert\data\raw\data_mail\sets\own_train_base.csv, Size: 5774
Test set saved: c:\Users\ilian\Documents\Projects\git_projects\university\phishing_bert\data\raw\data_mail\sets\own_test_base.csv, Size: 1444


In [85]:
df_own_train_base = pd.read_csv(own_train_base)
verify(df_own_train_base, "Own Train Base")

df_own_test_base = pd.read_csv(own_test_base)
verify(df_own_test_base, "Own Test Base")


Own Train Base, Rows: 5774
----------------------------------------
Class Distribution: {0: 2887, 1: 2887}
Language Distribution: {'de': 4981, 'en': 352, 'ru': 220, 'af': 28, 'sv': 19, 'fr': 18, 'unknown': 16, 'nl': 15, 'no': 15, 'et': 13, 'tl': 12, 'it': 10, 'mk': 10, 'id': 10, 'da': 10, 'bg': 8, 'tr': 6, 'cy': 6, 'pl': 5, 'es': 3, 'uk': 3, 'ca': 3, 'ro': 2, 'hu': 2, 'so': 2, 'fi': 2, 'ja': 1, 'pt': 1, 'lv': 1}
Detailed (Class, Language) Distribution: {(0, 'af'): 18, (0, 'ca'): 1, (0, 'cy'): 1, (0, 'da'): 9, (0, 'de'): 2528, (0, 'en'): 264, (0, 'et'): 13, (0, 'fi'): 1, (0, 'fr'): 12, (0, 'hu'): 2, (0, 'id'): 4, (0, 'it'): 2, (0, 'nl'): 4, (0, 'no'): 7, (0, 'ro'): 1, (0, 'sv'): 9, (0, 'tl'): 9, (0, 'tr'): 1, (0, 'unknown'): 1, (1, 'af'): 10, (1, 'bg'): 8, (1, 'ca'): 2, (1, 'cy'): 5, (1, 'da'): 1, (1, 'de'): 2453, (1, 'en'): 88, (1, 'es'): 3, (1, 'fi'): 1, (1, 'fr'): 6, (1, 'id'): 6, (1, 'it'): 8, (1, 'ja'): 1, (1, 'lv'): 1, (1, 'mk'): 10, (1, 'nl'): 11, (1, 'no'): 8, (1, 'pl'): 5, (1,

### Verify Balances

##### Verify Verification

In [ ]:
df_multilangual_curated_verification = pd.read_csv(os.path.join(data_mail_dir, "english_curated_verification.csv"))
verify(df_multilangual_curated_verification, "English Curated Verification")

df_multilangual_curated_verification = pd.read_csv(os.path.join(data_mail_dir, "german_curated_verification.csv"))
verify(df_multilangual_curated_verification, "German Curated Verification")

df_multilangual_curated_verification = pd.read_csv(os.path.join(data_mail_dir, "multilingual_curated_verification.csv"))
verify(df_multilangual_curated_verification, "Multilingual Curated Verification")

##### Verify English

In [13]:
# Curated-based train english
df_eng_curated_train = pd.read_csv(os.path.join(data_mail_dir, "english_curated_train.csv"))
verify(df_eng_curated_train, "English Curated Train")

# Curated-based test english
df_eng_curated_test = pd.read_csv(os.path.join(data_mail_dir, "english_curated_test.csv"))
verify(df_eng_curated_test, "English Curated Test")


English Curated Train, Rows: 20000
----------------------------------------
Class Distribution: {0: 10000, 1: 10000}
Language Distribution: {'en': 20000}
Detailed (Class, Language) Distribution: {(0, 'en'): 10000, (1, 'en'): 10000}
----------------------------------------

English Curated Test, Rows: 4000
----------------------------------------
Class Distribution: {0: 2000, 1: 2000}
Language Distribution: {'en': 4000}
Detailed (Class, Language) Distribution: {(0, 'en'): 2000, (1, 'en'): 2000}
----------------------------------------


##### Verify German

In [14]:
# Curated-based train german
df_germ_curated_train = pd.read_csv(os.path.join(data_mail_dir, "german_curated_train.csv"))
verify(df_germ_curated_train, "German Curated Train")

# Curated-based test german
df_germ_curated_test = pd.read_csv(os.path.join(data_mail_dir, "german_curated_test.csv"))
verify(df_germ_curated_test, "German Curated Test")


German Curated Train, Rows: 20000
----------------------------------------
Class Distribution: {0: 10000, 1: 10000}
Language Distribution: {'de': 20000}
Detailed (Class, Language) Distribution: {(0, 'de'): 10000, (1, 'de'): 10000}
----------------------------------------

German Curated Test, Rows: 4000
----------------------------------------
Class Distribution: {0: 2000, 1: 2000}
Language Distribution: {'de': 4000}
Detailed (Class, Language) Distribution: {(0, 'de'): 2000, (1, 'de'): 2000}
----------------------------------------


##### Verify Multi

In [15]:
# Curated-based train multi
df_mult_curated_train = pd.read_csv(os.path.join(data_mail_dir, "multilingual_curated_train.csv"))
verify(df_mult_curated_train, "Multilingual Curated Train")

# Curated-based test multi
df_mult_curated_test = pd.read_csv(os.path.join(data_mail_dir, "multilingual_curated_test.csv"))
verify(df_mult_curated_test, "Multilingual Curated Test")


Multilingual Curated Train, Rows: 20000
----------------------------------------
Class Distribution: {1: 10000, 0: 10000}
Language Distribution: {'en': 10000, 'de': 10000}
Detailed (Class, Language) Distribution: {(0, 'de'): 5000, (0, 'en'): 5000, (1, 'de'): 5000, (1, 'en'): 5000}
----------------------------------------

Multilingual Curated Test, Rows: 4000
----------------------------------------
Class Distribution: {1: 2000, 0: 2000}
Language Distribution: {'en': 2000, 'de': 2000}
Detailed (Class, Language) Distribution: {(0, 'de'): 1000, (0, 'en'): 1000, (1, 'de'): 1000, (1, 'en'): 1000}
----------------------------------------


##### Verify Own

In [16]:
# Own Base 
df_own_base = pd.read_csv(os.path.join(data_mail_dir, "own_base.csv"))
verify(df_own_base, "Own Base")

# Own Train
df_own_train_base = pd.read_csv(os.path.join(data_mail_dir, "own_train_base.csv"))
verify(df_own_train_base, "German Own Train")

# Own Test
df_own_test_base = pd.read_csv(os.path.join(data_mail_dir, "own_test_base.csv"))
verify(df_own_test_base, "German Own Test")


Own Base, Rows: 8162
----------------------------------------
Class Distribution: {1: 4543, 0: 3609, -1: 10}
Language Distribution: {'de': 7047, 'en': 470, 'ru': 335, 'af': 42, 'sv': 26, 'fr': 24, 'nl': 23, 'no': 21, 'unknown': 21, 'et': 18, 'bg': 16, 'tl': 15, 'it': 15, 'mk': 14, 'da': 13, 'id': 13, 'pl': 10, 'tr': 7, 'cy': 6, 'uk': 4, 'ca': 4, 'es': 4, 'so': 3, 'hu': 2, 'ro': 2, 'fi': 2, 'pt': 1, 'lv': 1, 'hr': 1, 'ja': 1, 'sk': 1}
Detailed (Class, Language) Distribution: {(-1, 'de'): 8, (-1, 'en'): 1, (-1, 'no'): 1, (0, 'af'): 23, (0, 'ca'): 1, (0, 'cy'): 1, (0, 'da'): 10, (0, 'de'): 3169, (0, 'en'): 323, (0, 'et'): 17, (0, 'fi'): 1, (0, 'fr'): 14, (0, 'hu'): 2, (0, 'id'): 5, (0, 'it'): 2, (0, 'nl'): 5, (0, 'no'): 8, (0, 'pl'): 1, (0, 'ro'): 1, (0, 'sk'): 1, (0, 'sv'): 11, (0, 'tl'): 12, (0, 'tr'): 1, (0, 'unknown'): 1, (1, 'af'): 19, (1, 'bg'): 16, (1, 'ca'): 3, (1, 'cy'): 5, (1, 'da'): 3, (1, 'de'): 3870, (1, 'en'): 146, (1, 'es'): 4, (1, 'et'): 1, (1, 'fi'): 1, (1, 'fr'): 10, (1